In [1]:
import numpy as np
from scipy.sparse import coo_matrix
np.random.seed(123)
np.set_printoptions(formatter={'float': '{:+.4e}'.format})

# $2 \times 2$ Example

In [2]:
a,b,c,d = 1,2,3,4
A = np.array([[a,b],[c,d]])
print(A)

[[1 2]
 [3 4]]


In [3]:
def compang(a,b,c,d):
    x = np.atan2(b-c,a+d)
    y = np.atan2(b+c,a-d)
    return (x-y)/2, (x+y)/2

In [4]:
theta, phi = compang(a,b,c,d)

In [5]:
Jtheta = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta),np.cos(theta)]])
Jphi = np.array([[np.cos(phi), -np.sin(phi)],[np.sin(phi),np.cos(phi)]])

In [6]:
Jtheta @ A @ Jphi

array([[+5.4650e+00, +1.2507e-16],
       [+1.7136e-16, -3.6597e-01]])

# $n \times n$ Example

In [7]:
def sparserot(n,i,j,theta):
    rows = np.arange(n+2)
    cols = np.arange(n+2)
    values = np.ones(n+2)
    
    values[i] = np.cos(theta)
    values[j] = np.cos(theta)
    values[n] = -np.sin(theta)
    values[n+1] = np.sin(theta)

    rows[n] = i
    cols[n] = j
    
    rows[n+1] = j
    cols[n+1] = i

    return coo_matrix((values, (rows, cols)), shape=(n, n))

In [8]:
theta = np.pi/6

In [9]:
Jtheta = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta),np.cos(theta)]])
print(Jtheta)

[[+8.6603e-01 -5.0000e-01]
 [+5.0000e-01 +8.6603e-01]]


In [10]:
Jtheta = sparserot(2,0,1,theta)
print(Jtheta.toarray())

[[+8.6603e-01 -5.0000e-01]
 [+5.0000e-01 +8.6603e-01]]


In [11]:
Jtheta = sparserot(5,1,3,theta)
print(Jtheta.toarray())

[[+1.0000e+00 +0.0000e+00 +0.0000e+00 +0.0000e+00 +0.0000e+00]
 [+0.0000e+00 +8.6603e-01 +0.0000e+00 -5.0000e-01 +0.0000e+00]
 [+0.0000e+00 +0.0000e+00 +1.0000e+00 +0.0000e+00 +0.0000e+00]
 [+0.0000e+00 +5.0000e-01 +0.0000e+00 +8.6603e-01 +0.0000e+00]
 [+0.0000e+00 +0.0000e+00 +0.0000e+00 +0.0000e+00 +1.0000e+00]]


In [12]:
A = np.random.randn(5,5)
print(A)

[[-1.0856e+00 +9.9735e-01 +2.8298e-01 -1.5063e+00 -5.7860e-01]
 [+1.6514e+00 -2.4267e+00 -4.2891e-01 +1.2659e+00 -8.6674e-01]
 [-6.7889e-01 -9.4709e-02 +1.4914e+00 -6.3890e-01 -4.4398e-01]
 [-4.3435e-01 +2.2059e+00 +2.1868e+00 +1.0041e+00 +3.8619e-01]
 [+7.3737e-01 +1.4907e+00 -9.3583e-01 +1.1758e+00 -1.2539e+00]]


In [13]:
U,S,V = np.linalg.svd(A)
print(S)

[+4.4670e+00 +2.9873e+00 +2.3767e+00 +1.5391e+00 +7.8710e-02]


In [14]:
def jacobirot(A,i,j):

    assert A.shape[0]==A.shape[1]
    n = A.shape[0]
    
    a = A[i,i]
    b = A[i,j]
    c = A[j,i]
    d = A[j,j]
    
    theta, phi = compang(a,b,c,d)
    
    Jtheta = sparserot(n,i,j,theta)
    Jphi= sparserot(n,i,j,phi)

    return Jtheta, Jphi

In [15]:
Jtheta, Jphi = jacobirot(A,1,3)

In [16]:
Jtheta @ A @ Jphi

array([[-1.0856e+00, +1.1480e+00, +2.8298e-01, -1.3949e+00, -5.7860e-01],
       [-1.5518e+00, +3.2930e+00, +1.7194e+00, -7.7716e-16, +9.1489e-01],
       [-6.7889e-01, -2.8044e-02, +1.4914e+00, -6.4527e-01, -4.4398e-01],
       [-7.1253e-01, -1.1657e-15, -1.4176e+00, -1.5879e+00, +2.5170e-01],
       [+7.3737e-01, +1.3610e+00, -9.3583e-01, +1.3239e+00, -1.2539e+00]])

In [17]:
n = A.shape[0]
for i in range(n):
    for j in range(i+1,n):
        Jtheta, Jphi = jacobirot(A,i,j)
        A = Jtheta @ A @ Jphi
print(A)

[[+4.3480e+00 +3.7937e-01 +1.9904e-01 -6.0777e-02 +7.8149e-01]
 [+2.0078e-02 +2.1662e+00 -4.1094e-01 +5.3964e-02 +1.2588e+00]
 [+2.0375e-02 -6.7008e-02 +2.8194e+00 +6.7542e-03 -1.8228e-01]
 [-7.2133e-02 +3.0132e-01 +5.6634e-02 +1.5659e+00 +1.7347e-16]
 [+7.8010e-02 -4.1205e-01 -1.3649e-01 +4.3368e-18 -1.2523e-01]]


In [18]:
n = A.shape[0]
for i in range(10):
    for i in range(n):
        for j in range(i+1,n):
            Jtheta, Jphi = jacobirot(A,i,j)
            A = Jtheta @ A @ Jphi
print(A)

[[+4.4670e+00 +1.3309e-226 +1.6754e-231 -6.0343e-231 -7.3864e-127]
 [-6.8791e-136 +2.9873e+00 +1.6958e-167 -3.0728e-234 +1.5275e-287]
 [+1.5490e-120 -3.4493e-235 +2.3767e+00 -4.6772e-239 +0.0000e+00]
 [-1.2050e-181 +1.4917e-154 -1.1162e-239 +1.5391e+00 +0.0000e+00]
 [-7.6172e-127 -9.8619e-132 -7.4165e-136 -2.5797e-136 +7.8710e-02]]


In [19]:
S0 = np.sort(np.abs(np.diag(A)))[::-1]

In [20]:
print(S0)

[+4.4670e+00 +2.9873e+00 +2.3767e+00 +1.5391e+00 +7.8710e-02]


In [21]:
err = np.abs(S - S0)
print(err)

[+2.6645e-15 +1.7764e-15 +0.0000e+00 +1.1102e-15 +3.7470e-16]


# 